# Goal: 

The goal of this notebook is to try a bunch of the Scikit learn APIs on a bunch of rudimentary features extracted from the images and see which algorithm does the best. Then, using that algorithm, we perform ablation studies to see how each set of features affect the performance of the Scikit learn regression API.


## Imports

In [1]:
import numpy as np
import random
import pandas as pd
from PIL import Image
from scipy.stats import spearmanr, pearsonr
from sklearn.linear_model    import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import (
    RandomForestRegressor, ExtraTreesRegressor,
    HistGradientBoostingRegressor,
    AdaBoostRegressor, BaggingRegressor,
    GradientBoostingRegressor
)
from sklearn.svm             import SVR
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.neural_network  import MLPRegressor
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from skimage.color   import rgb2lab, rgb2hsv
from scipy import ndimage
import pywt
SEED = 42
random.seed(SEED)
np.random.seed(SEED)


## Reading the preparing the data

In [2]:
train_df = pd.read_csv("../../dataset/ratings/train.csv")
test_df = pd.read_csv("../../dataset/ratings/test.csv")


In [3]:
    val_df = train_df.iloc[-90:].copy()
train_df = train_df.iloc[:-90].copy()
print(f"val_df.shape: {val_df.shape}")
print(f"train_df.shape: {train_df.shape}")


val_df.shape: (90, 3)
train_df.shape: (420, 3)


In [4]:
train_df.head()


,filename,MOS_Rating,MOS_ZScore
0,f329.png,66.346670,0.134646
1,f316.png,47.456209,-0.954147
2,f257.png,80.986020,0.978417
3,f206.png,11.237613,-3.041686
4,f459.png,60.666032,-0.192770


In [5]:
val_df.head()


,filename,MOS_Rating,MOS_ZScore
420,f68.png,61.416769,-0.149500
421,f191.png,59.814871,-0.241829
422,f278.png,66.696317,0.154798
423,f62.png,72.154690,0.469404
424,f164.png,75.020202,0.634564


In [6]:
test_df.head()


,filename,MOS_Rating,MOS_ZScore
0,f244.png,75.843124,0.681995
1,f126.png,76.323762,0.709697
2,r3.png,85.236324,1.223393
3,f145.png,42.999260,-1.211034
4,f170.png,51.925539,-0.696548


In [7]:
print(f"train_df.shape: {train_df.shape}")
print(f"val_df.shape: {val_df.shape}")
print(f"test_df.shape: {test_df.shape}")


train_df.shape: (420, 3)
val_df.shape: (90, 3)
test_df.shape: (90, 3)


In [8]:
train_files = set(train_df['filename'])
val_files   = set(val_df['filename'])
test_files  = set(test_df['filename'])

train_val_overlap   = train_files.intersection(val_files)
train_test_overlap  = train_files.intersection(test_files)
val_test_overlap    = val_files.intersection(test_files)

# Print summary and details if any overlaps exist
print(f"Overlap between train and val: {len(train_val_overlap)} file(s)")
if train_val_overlap:
    print("Files:", train_val_overlap)

print(f"Overlap between train and test: {len(train_test_overlap)} file(s)")
if train_test_overlap:
    print("Files:", train_test_overlap)

print(f"Overlap between val and test: {len(val_test_overlap)} file(s)")
if val_test_overlap:
    print("Files:", val_test_overlap)


Overlap between train and val: 0 file(s)
Overlap between train and test: 0 file(s)
Overlap between val and test: 0 file(s)


In [9]:
X_train = list(train_df["filename"])
y_train = list(train_df["MOS_ZScore"])

X_valid = list(val_df["filename"])
y_valid = list(val_df["MOS_ZScore"])

X_test = list(test_df["filename"])
y_test = list(test_df["MOS_ZScore"])


## Defining `FeatureExtractor`:

In [10]:
class FeatureExtractor:
    def __init__(self, img_dir=""):
        self.img_dir = img_dir

    def extract(self, image_name):
        img = np.asarray(Image.open(self.img_dir + image_name))
        gray = np.mean(img, axis=2)
        h, w = gray.shape

        feats = []


        # — Color features
        lab = rgb2lab(img)
        hsv = rgb2hsv(img)
        lab_a = lab[:, :, 1].astype(np.float32)/127
        lab_b = lab[:, :, 2].astype(np.float32)/127
        feats.extend([
            np.std(lab_a), np.std(lab_b),
            hsv[:,:,0].std(),
            np.percentile(hsv[:,:,1],95),
            np.mean(np.abs(lab_a)), np.mean(np.abs(lab_b)),
            np.median(hsv[:,:,2]),
            (hsv[:,:,2]>0.9).mean()
        ])



        # — Edge features
        sx = ndimage.sobel(gray, axis=1)
        sy = ndimage.sobel(gray, axis=0)
        mag = np.hypot(sx,sy)
        feats.extend([
            mag.mean(), mag.std(),
            np.percentile(mag,90),
            (mag>0.2).mean(),
            np.corrcoef(sx[::10].flatten(), sy[::10].flatten())[0,1],
            np.std(np.arctan2(sy,sx)[mag>0.1]),
            np.abs(sx - np.fliplr(sx)).mean(),
            np.abs(sy - np.flipud(sy)).mean(),
            np.mean(mag[mag>0.05]), mag.max()
        ])


        # — Texture (GLCM + HOG)
        glcm = graycomatrix((gray*255).astype(np.uint8),
                            distances=[1,3],
                            angles=[0,np.pi/4],
                            levels=256, symmetric=True)
        for prop in ["contrast","dissimilarity","homogeneity","energy","correlation"]:
            feats.extend(graycoprops(glcm, prop).flatten())
        hogf = hog(gray, orientations=8,
                   pixels_per_cell=(32,32),
                   cells_per_block=(1,1),
                   visualize=False)
        feats.extend([hogf.mean(), hogf.std()])



        # — Frequency (wavelets)
        coeffs = pywt.wavedec2(gray, 'db2', level=3)
        for cH,cV,cD in coeffs[1:]:
            feats.extend([
                np.abs(cH).mean(),
                np.abs(cV).mean(),
                np.abs(cD).mean(),
                np.percentile(np.abs(cH), 90)
            ])



        # — Perceptual grouping
        ce = ndimage.gaussian_filter(gray,1) - ndimage.gaussian_filter(gray,3)
        feats.append(np.mean(ce**2))
        sal = np.abs(ndimage.gaussian_gradient_magnitude(gray,2))
        feats.extend([sal.mean(), sal.std()])

        return np.array(feats)

    def transform(self, image_list):
        return np.vstack([self.extract(img) for img in image_list])


### Extract features for train/val/test


In [11]:
fe = FeatureExtractor(img_dir="../../dataset/images/train_images/")
X_train_feats = fe.transform(X_train)
X_valid_feats = fe.transform(X_valid)



In [12]:
fe = FeatureExtractor(img_dir="../../dataset/images/test_images/")
X_test_feats  = fe.transform(X_test)


## Experiment with a suite of regressors


### Train and Val

In [13]:
models = {
    "LinearRegression":      LinearRegression(),
    "Ridge":                 Ridge(alpha=1.0, random_state=SEED),
    "Lasso":                 Lasso(alpha=0.1, random_state=SEED),
    "ElasticNet":            ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=SEED),
    "DecisionTree":          DecisionTreeRegressor(max_depth=5, random_state=SEED),
    "RandomForest":          RandomForestRegressor(n_estimators=100, random_state=SEED),
    "ExtraTrees":            ExtraTreesRegressor(n_estimators=100, random_state=SEED),
    "HistGB":                HistGradientBoostingRegressor(max_iter=100, random_state=SEED),
    # "AdaBoost":              AdaBoostRegressor(n_estimators=100, random_state=SEED),
    "AdaBoost": AdaBoostRegressor(
    estimator=DecisionTreeRegressor(random_state=SEED), 
    n_estimators=100, 
    random_state=SEED
    ),
    # "Bagging":               BaggingRegressor(n_estimators=50, random_state=SEED),    
    "Bagging": BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=SEED), 
    n_estimators=50, 
    random_state=SEED
    ),
    
    "SVR":                   SVR(kernel="rbf", C=1.0, epsilon=0.1),
    "KNN":                   KNeighborsRegressor(n_neighbors=5),
    "MLP":                   MLPRegressor(hidden_layer_sizes=(100,50), max_iter=200, random_state=SEED),
    "GBR":                   GradientBoostingRegressor(n_estimators=300, learning_rate=0.1,
                                                        max_depth=5, random_state=SEED),
}







# -------------------------------------------------------------------
#  4) Experiment: train on train, validate on val, pick best, test on test
# -------------------------------------------------------------------
best_name    = None
best_spearman = -np.inf
best_model   = None
results = {}

for name, model in models.items():
    # 1) fit on train
    model.fit(X_train_feats, y_train)

    # 2) score on val
    preds_val = model.predict(X_valid_feats)
    spearman_val, _ = spearmanr(preds_val, y_valid)
    pearson_val,  _ = pearsonr(preds_val, y_valid)

    results[name] = {"spearman": spearman_val, "pearson": pearson_val}
    print(f"{name:20s} → Spearman: {spearman_val:.4f}, Pearson: {pearson_val:.4f}")

    # 3) track best by Spearman on val
    if spearman_val > best_spearman:
        best_spearman = spearman_val
        best_name     = name
        best_model    = model

print(f"\nBest on val → {best_name} (Spearman {best_spearman:.4f})")


LinearRegression     → Spearman: 0.3428, Pearson: 0.3178
Ridge                → Spearman: 0.3658, Pearson: 0.3824
Lasso                → Spearman: 0.4628, Pearson: 0.4237
ElasticNet           → Spearman: 0.4353, Pearson: 0.4007
DecisionTree         → Spearman: 0.5331, Pearson: 0.4895


/Users/annimukh/miniconda3/envs/image_realness/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.157e-01, tolerance: 4.190e-02
  model = cd_fast.enet_coordinate_descent(
/Users/annimukh/miniconda3/envs/image_realness/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.246e+01, tolerance: 4.190e-02
  model = cd_fast.enet_coordinate_descent(


RandomForest         → Spearman: 0.4660, Pearson: 0.4381
ExtraTrees           → Spearman: 0.4168, Pearson: 0.3861
HistGB               → Spearman: 0.4856, Pearson: 0.4062
AdaBoost             → Spearman: 0.4397, Pearson: 0.3477
Bagging              → Spearman: 0.5090, Pearson: 0.4462
SVR                  → Spearman: 0.2796, Pearson: 0.3081
KNN                  → Spearman: 0.3574, Pearson: 0.3701
MLP                  → Spearman: -0.0848, Pearson: 0.0615
GBR                  → Spearman: 0.4901, Pearson: 0.4622

Best on val → DecisionTree (Spearman 0.5331)


### Test

In [14]:
# -------------------------------------------------------------------
#  5) Evaluate the selected best model on test set
# -------------------------------------------------------------------
preds_test = best_model.predict(X_test_feats)
spearman_test, _ = spearmanr(preds_test, y_test)
pearson_test,  _ = pearsonr(preds_test, y_test)

print(f"Test performance of {best_name}:")
print(f"  Spearman: {spearman_test:.4f}")
print(f"  Pearson : {pearson_test:.4f}")


Test performance of DecisionTree:
  Spearman: 0.5033
  Pearson : 0.4339
